In [2]:
import pandas as pd

In [9]:
public_transport = pd.read_csv('datasets/Daily_Public_Transport_Passenger_Journeys_by_Service_Type_20250830.csv')
public_transport.head()

,Date,Local Route,Light Rail,Peak Service,Rapid Route,School,Other
0,30/08/2024,"16,436","10,705",225,"19,026","3,925",59
1,15/09/2023,"15,499","10,671",267,"18,421","4,519",61
2,28/12/2021,"1,756","2,352",0,"3,775",0,13
3,11/01/2023,"10,536","8,347",223,"14,072",0,48
4,11/09/2021,820,612,0,"1,283",0,11


In [3]:
school_bus = pd.read_csv('datasets/ACT_School_Bus_Services.csv')
school_bus.head()

,RouteNumber,Shift,StartTime,VehicleStyle,Description,Address,Phone,School Name,Website,Location
0,1001,1550,7:53:00 AM,Euro,"Forde, Bonner, Amaroo, Ngunnawal","1021 Gungahlin Dr, Nicholls",(02) 6163 4800,St John Paul II College,https://www.transport.act.gov.au/getting-aroun...,POINT (149.1008999 -35.17702592)
1,1002,1041,8:33:00 AM,IRIS,Gungahlin Place Non Stop,"1021 Gungahlin Dr, Nicholls",(02) 6163 4800,St John Paul II College,https://www.transport.act.gov.au/getting-aroun...,POINT (149.1008999 -35.17702592)
2,1009,1034,7:23:00 AM,PR 3,"Ngunnawal, Moncrieff, Amaroo, Bonner","4 The Valley Avenue, Gungahlin",(02) 6255 7700,Burgmann Anglican School Valley Campus,https://www.transport.act.gov.au/getting-aroun...,POINT (149.1236662 -35.18784524)
3,1001,1358,8:05:00 AM,Euro,"Bonner, Amaroo, Ngunnawal","1021 Gungahlin Dr, Nicholls",(02) 6163 4800,St John Paul II College,https://www.transport.act.gov.au/getting-aroun...,POINT (149.1008999 -35.17702592)
4,1002,1605,3:44:00 PM,Euro,Gungahlin Place Non Stop,"1021 Gungahlin Dr, Nicholls",(02) 6163 4800,St John Paul II College,https://www.transport.act.gov.au/getting-aroun...,POINT (149.1008999 -35.17702592)


In [62]:
school_names = school_bus['School Name'].unique()
school_names

array(['St John Paul II College',
       'Burgmann Anglican School Valley Campus',
       'Emmaus Christian School', 'Daramalan College',
       'Marist College Canberra', 'Melba Copland Secondary School 11-12',
       "St Thomas More's Primary School", 'Trinity Christian School',
       'Melrose High School', 'Canberra Grammar Junior School',
       'St Mary Mackillop College - Senior Campus',
       'Lyneham Primary School',
       'St Mary Mackillop College - Junior Campus', 'Radford College',
       'St Francis Xavier College', "St Clare's College",
       'Forrest Primary School', 'Alfred Deakin High School',
       "St Anthony's Parish Primary School",
       "St Matthew's Primary School",
       'Canberra Girls Grammar Junior School', 'Orana Steiner School',
       'Telopea Park School P-6', "St Edmund's College Canberra",
       'Canberra Montessori School', 'Merici College',
       'Melba Copland Secondary School 7-10', 'Calwell High School',
       'Charnwood-Dunlop School', 

In [ ]:
sc

In [13]:
bus_route = pd.read_csv('datasets/Bus_Routes.csv')
bus_route.head()

,the_geom,Id,shape_id,route_id,short_name,long_name,route_desc,route_type,color,text_color,route_url,rt_tp_txt
0,MULTILINESTRING ((149.12561599999998 -35.27913...,0,100119,10-10516,10,Denman Prospect to City via John Gorton Dr,NaN,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus
1,"MULTILINESTRING ((149.022007 -35.299196, 149.0...",0,100121,10-10516,10,Denman Prospect to City via John Gorton Dr,NaN,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus
2,MULTILINESTRING ((149.12561599999998 -35.27913...,0,1800014,180-10516,180,Lanyon to City West via Condor & Banks,NaN,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus
3,MULTILINESTRING ((149.09249400000002 -35.45621...,0,1800018,180-10516,180,Lanyon to City West via Condor & Banks,NaN,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus
4,"MULTILINESTRING ((149.135127 -35.185345, 149.1...",0,180104,18-10516,18,Mitchell Loop via Harrison & Franklin,NaN,3,0042AC,FFFFFF,http://www.transport.act.gov.au/routes-and-tim...,Bus


In [17]:
school_distance = pd.read_csv('datasets/Students_Distance_from_School_-_by_school_and_SA1_20250830.csv')
school_distance.head()

,School,SA1,No. of Students
0,Ainslie School,80105105102,5
1,Ainslie School,80105105302,5
2,Ainslie School,80105105404,5
3,Ainslie School,80105105605,5
4,Ainslie School,80105105017,6


In [47]:
school_distance.groupby(['School', 'SA1'])['No. of Students'].sum().sort_values(ascending=False)

School                     SA1        
Garran Primary School      80109110207    128
Amaroo School              80104111801    117
                           80104103410     80
                           80104112004     75
                           80104103405     73
                                         ... 
Gordon Primary School      80107107505      5
Stromlo High School        80107108213      5
                           80107108212      5
Gold Creek School          80104104811      5
Yarralumla Primary School  80107108235      5
Name: No. of Students, Length: 2245, dtype: int64

In [19]:
park_ride = pd.read_csv('datasets/Park_And_Ride_Locations.csv')
park_ride.head()

,Suburb,Location,Point,Permit Type
0,Belconnen,"Belconnen Community Bus Station, access from S...","(-35.239602, 149.069423)",Permit Only
1,Calwell,Calwell Shops - Webber Street,"(-35.433428, 149.113051)",No Permit Required
2,Kambah,Kambah Village Shops - Marconi Crescent,"(-35.379524, 149.057408)",No Permit Required
3,Weston,North Weston - Kirkpatrick Street,"(-35.320885, 149.058926)",No Permit Required
4,Belconnen,Corner of Josephson Street and Walder Street,"(-35.23824, 149.060841)",Permit Only


In [21]:
student_population = pd.read_csv('datasets/Census_Data_for_all_ACT_Schools_20250830.csv')
student_population.head()

,Census,School Name,Category,Year Level,Students
0,14 February 2025,Ainslie School,Gov,Year 2,60
1,14 February 2025,Ainslie School,Gov,Kindergarten,50
2,14 February 2025,Ainslie School,Gov,Year 5,45
3,14 February 2025,Ainslie School,Gov,Year 4,45
4,14 February 2025,Ainslie School,Gov,Year 3,51


In [33]:
student_population.tail()

,Census,School Name,Category,Year Level,Students
27312,01 February 2009,Yarralumla Primary School,Gov,Year 3,29
27313,01 February 2009,Yarralumla Primary School,Gov,Year 4,22
27314,01 February 2009,Yarralumla Primary School,Gov,Year 5,28
27315,01 February 2009,Yarralumla Primary School,Gov,Year 6,24
27316,01 February 2009,Yarralumla Primary School,Gov,Preschool,56


In [60]:
student_population['School Name'].nunique()

144

In [41]:
student_population.groupby('School Name')['Students'].sum().sort_values(ascending=False)

School Name
Amaroo School                      57231
Harrison School                    47916
Telopea Park School                44793
Gold Creek School                  39828
Lyneham High School                35001
                                   ...  
Communities@Work Galilee School      700
Shirley Smith High School            346
Woden School, The                     93
Aunty Agnes Shea High School          81
Canberra Jewish School                18
Name: Students, Length: 144, dtype: int64

In [25]:
population_projection = pd.read_excel('datasets/ACT-Population-Projections-2022-2060-Datasheet.xlsx')
population_projection.head()

,Unnamed: 0,Unnamed: 1,ACT Population Projections: 2022 - 2060
0,List of Tables,NaN,NaN
1,Table 1,NaN,"Summary, 2022 - 2060"
2,Table 2,NaN,"Projections by SA3 (District Level), 2022 - 2060"
3,Table 3,NaN,Projections by single year of age and by SA3 -...
4,Table 4,NaN,Projections by single year of age and by SA3 -...


In [27]:
population_projection.tail()

,Unnamed: 0,Unnamed: 1,ACT Population Projections: 2022 - 2060
9,Table 9,NaN,Projections by single year of age and by SA2 -...
10,Table 10,NaN,"Projections by single year of age - Persons, 2..."
11,Table 11,NaN,"Projections by single year of age - Males, 202..."
12,Table 12,NaN,"Projections by single year of age - Females, 2..."
13,Contact and Disclaimer,NaN,Contact and Disclaimer


In [31]:
national_roads = pd.read_csv('datasets/National_Roads_-1036766347759299072.csv')
national_roads.head()

/var/folders/wf/dm__wbb55gsf9_40cvpwfz1r0000gn/T/ipykernel_6224/4188913487.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  national_roads = pd.read_csv('datasets/National_Roads_-1036766347759299072.csv')


,Road id,Contributor id,Jurisdiction control,Operator,Date created,Date modified,National Route,State Route,Full Street Name,Street name,...,Surface,Trafficability,Travel direction,Speed,State,Source,Horizontal accuracy,OBJECTID,GlobalID,Shape Length
0,rds87ca8757d73a,16015103990000325104,NaN,NaN,3/25/2025 12:00:00 AM,NaN,NaN,NaN,ST JAMES DRIVE,ST JAMES,...,SEALED,NaN,NaN,NaN,QLD,QLD,NaN,1,82abee34-f73b-4073-b1e2-ab5c3d5575f9,0.001146
1,rdslU92OKL4dQeF,518468856,NaN,NaN,3/25/2025 12:00:00 AM,NaN,NaN,NaN,EASTERN CIRCUIT,EASTERN,...,SEALED,2WD,NaN,20.0,NSW,NSW,10.0,2,86ab6cdf-b876-48a9-87d9-447b0f4253cd,0.000749
2,rdsad4aede16501,515848696,NaN,NaN,3/25/2025 12:00:00 AM,NaN,NaN,NaN,THE BOULEVARD,THE BOULEVARD,...,SEALED,2WD,NaN,100.0,NSW,NSW,10.0,3,d91e598f-01c7-4256-a95f-8b7d8b40683f,0.000018
3,rds6c1a711dfd1d,517569409,Transport for New South Wales (controlled roads),NaN,3/25/2025 12:00:00 AM,NaN,A1,NaN,PACIFIC HIGHWAY,PACIFIC,...,SEALED,2WD,NaN,60.0,NSW,NSW,5.0,4,a2749c84-db27-41b8-b6fa-d1970eb4c04c,0.000353
4,rds19943e60fcbd,16015103990000025963,TMR,NaN,1/21/2025 12:00:00 AM,NaN,NaN,90,BEAUDESERT NERANG ROAD,BEAUDESERT NERANG,...,SEALED,2WD,BOTH,NaN,QLD,QLD,NaN,5,7808dbaf-5917-4ec3-bbda-499c8fb04e6e,0.002064
